In [8]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import databaseconfig as cfg
import sqlalchemy as db
import psycopg2
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression

In [2]:
# Connect to database
engine = db.create_engine(f"postgresql+psycopg2://{cfg.user}:{cfg.password}@{cfg.host}:{cfg.port}/{cfg.database}")
metadata = db.MetaData()
connection = engine.connect()

In [6]:
# Read data
installations = db.Table('combined_data',metadata,autoload=True,autoload_with=engine)
installation_results = connection.execute(db.select([installations])).fetchall()
installations_df= pd.DataFrame(installation_results)
installations_df.columns = installation_results[0].keys()
installations_df=installations_df.sort_values('installation_date')

In [10]:
n_samples = len(installations_df)
print(n_samples)
installations_df.head()

139002


,system_id,data_provider,provider,installation_date,install_yr,system_size,system_inverter_capacity,total_installed_price,appraised_value_flag,rebate_or_grant,...,module_model_4,bipv_module,inverter_manufacturer,inverter_mfg,inverter_country,inverter_region,inverter_model_1,inverter_loading_ratio,microinverter_1,dc_optimizer
120591,AZ_rTd8MUXNdj,1,Arizona Public Service,1998-04-01,1998.0,110.30,NaN,NaN,0,NaN,...,NaN,NaN,NaN,None,None,None,None,NaN,NaN,0
28048,AZ_zL1ZsoLdc1,1,Arizona Public Service,1999-06-01,1999.0,24.70,NaN,NaN,0,NaN,...,NaN,NaN,NaN,None,None,None,None,NaN,NaN,0
95851,AZ_37qZr8MKLC,10,Tucson Electric Power,1999-09-09,1999.0,5.20,NaN,25000.0,0,NaN,...,NaN,NaN,NaN,None,None,None,None,NaN,NaN,0
98784,AZ_8kXakccpQz,10,Tucson Electric Power,1999-09-09,1999.0,1.45,NaN,6095.0,0,NaN,...,NaN,NaN,NaN,None,None,None,None,NaN,NaN,0
104227,AZ_dZ0CK8ChgY,10,Tucson Electric Power,1999-09-09,1999.0,2.55,NaN,11220.0,1,NaN,...,NaN,NaN,NaN,None,None,None,None,NaN,NaN,0


In [ ]:
y = 